### 04. RAG med Vector Databas

- mer avancerade uppslag av data som sedan skickas med i prompten

- todo: förklara grovt vad en vector db är ??

![04 diagram](docs/04.drawio.png)

In [6]:
import os
from dotenv import load_dotenv

from google import genai  # client for google gemini api
from google.genai import types

import chromadb

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")  # load the api key from another file (.env)

client = genai.Client(api_key=API_KEY)  # create an instance of the client

In [9]:
# ladda vector databas
chroma_client = chromadb.PersistentClient(path="data/vector_db")
collection = chroma_client.get_or_create_collection(name="some_data")

# ladda pdf-filer
loader = PyPDFDirectoryLoader("data/documents")
raw_documents = loader.load()


# splitta filer i delar
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

chunks = text_splitter.split_documents(raw_documents)

# preparing to be added in chromadb
documents = []
metadata = []
ids = []

i = 0

for chunk in chunks:
    documents.append(chunk.page_content)
    ids.append("ID"+str(i))
    metadata.append(chunk.metadata)

    i += 1

# adding to chromadb
collection.upsert(
    documents=documents,
    metadatas=metadata,
    ids=ids
)

C:\Users\pedhe\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:07<00:00, 10.8MiB/s]


In [11]:
results = collection.query(
    query_texts=["hur lång tid kommer genomförandet av spoe-projektet att pågå"],
    n_results=4
)

print(results['documents'])

[['inte att använda förrän ServiceCenter genomfört kontroll. Efter kontroll frisläpps ny kund enligt ordinarie rutin i D365. •Om loggning finns inbyggt i ramverket så ingår det annars måste det utredasEstimat för MVP 1; 160 tSEK (inkluderat i nuvarande budget)Genomförande under 5-6 veckor.   *MVP är', 'SPOE - Omfattning för MVP 1MVP*  – En första körbar produkt med nedan innehåll: •En PowerApp för att skapa ny kund i D365 (F&O Kundmasterdata koncerngemensamma, 12-15 attribut som används idag av ServiceCenter) Wood Sverige.•Möjlighet att göra val från DropDown med live-data från D365.•Validering av', 'förvaltningen och sedan vid behov vidare till GST förvaltningen.• Vidareutveckling initieras av produktägaren‒ Team: wipro och representant från superuser grupp‒ Vid stor förändring initiera projektteam• Testning:‒ Utv tester - Wipro‒ Acc tester - Superuser gruppen + ev utvalda säljare2021-04-26 15', 'Wood Sverige.•Möjlighet att göra val från DropDown med live-data från D365.•Validering av

In [ ]:
system_prompt = """
You are a helpful assistant. You answer questions about the SPOE project. 
But you only answer based on knowledge I'm providing you. You don't use your internal 
knowledge and you don't make things up. If you don't know the answer, just say: I don't know
--------------------
The data:
"""+str(results['documents'])+"""
"""

# concat to array with qurty -> system -> user


response = client.models.generate_content(
    model="gemini-2.0-flash", contents="tell me one funny joke, max 100 words"
)

# skriv ut svar från api
print(response.candidates[0].content.parts[0].text)